In [ ]:
import numpy as np
import xarray as xr
from glob import glob
import cftime

In [ ]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=1, threads_per_worker=4, memory_limit="32G")
client = Client(cluster)
client

In [ ]:
for y in np.arange(1996, 2016):
    year = str(y)
    print(year)
    data_filesREF_gridU = sorted(glob("/data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-REF08-S/5d/" + year + "/CREG12.L75-REF08_*.5d_gridU.nc"))
    data_filesREF_gridV = sorted(glob("/data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-REF08-S/5d/" + year + "/CREG12.L75-REF08_*.5d_gridV.nc"))
    UREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridU, coords="minimal", compat="override", parallel=True, decode_cf=False))
    VREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridV, coords="minimal", compat="override", parallel=True, decode_cf=False))
    uUREF = UREF.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
    vVREF = VREF.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
    dudzREF = uUREF.differentiate("deptht")
    dvdzREF = vVREF.differentiate("deptht")
    shearREF = ((dudzREF**2) + (dvdzREF**2)).to_dataset(name="vovshear")
    days, datasets = zip(*shearREF.groupby("time_counter"))
    datasets = list(datasets)
    datasets = [datasets[j].expand_dims(dim={"time_counter": 1}) for j in np.arange(0, len(datasets))]
    dates = list(xr.cftime_range(start=year + "-01-05", end=year + "-12-31", freq="5D", calendar="noleap"))
    if year in ["2001", "2014", "2015"]:
        dates = dates[0:36] + list(xr.cftime_range(start=str(days[0].year) + "-07-05", end=str(days[0].year) + "-12-31", freq="5D", calendar="noleap"))
    paths = [str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-shear/" + year + "/CREG12.L75-REF08_y" 
                 + "%04d" % d.year + "m" + "%02d" % d.month + "d" + "%02d" % d.day + ".5d_shear.nc") for d in dates]
    xr.save_mfdataset(datasets, paths, engine="netcdf4", format="NETCDF4")

In [ ]:
print("clim")
data_filesREF_gridU = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m*_gridUclim.nc"))
data_filesREF_gridV = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-UV_clim/CREG12.L75-REF08_m*_gridVclim.nc"))
UREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridU, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
VREF = xr.decode_cf(xr.open_mfdataset(data_filesREF_gridV, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
uUREF = UREF.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
vVREF = VREF.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
dudzREF = uUREF.differentiate("deptht")
dvdzREF = vVREF.differentiate("deptht")
shearREF = ((dudzREF**2) + (dvdzREF**2)).to_dataset(name="vovshear")
path = str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-REF08-shear/clim/CREG12.L75-REF08_1996-2015_climatology_vovshear.nc")
shearREF.to_netcdf(path, engine="netcdf4", format="NETCDF4")

In [ ]:
for y in np.arange(2051, 2071):
    year = str(y)
    print(year)
    data_filesFUT_gridU = sorted(glob("/data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-FUT08-S/5d/" + year + "/CREG12.L75-FUT08_*.5d_gridU.nc"))
    data_filesFUT_gridV = sorted(glob("/data0/project/drakkar/CONFIGS/CREG12.L75/CREG12.L75-FUT08-S/5d/" + year + "/CREG12.L75-FUT08_*.5d_gridV.nc"))    
    UFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridU, coords="minimal", compat="override", parallel=True, decode_cf=False))
    VFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridV, coords="minimal", compat="override", parallel=True, decode_cf=False))
    uUFUT = UFUT.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
    vVFUT = VFUT.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
    dudzFUT = uUFUT.differentiate("deptht")
    dvdzFUT = vVFUT.differentiate("deptht") 
    shearFUT = ((dudzFUT**2) + (dvdzFUT**2)).to_dataset(name="vovshear")
    days, datasets = zip(*shearFUT.groupby("time_counter"))
    datasets = list(datasets)
    datasets = [datasets[j].expand_dims(dim={"time_counter": 1}) for j in np.arange(0, len(datasets))]
    dates = list(xr.cftime_range(start=year + "-01-05", end=year + "-12-31", freq="5D", calendar="noleap"))
    if year in ["2063"]:
        dates = dates[0:36] + list(xr.cftime_range(start=str(days[0].year) + "-07-05", end=str(days[0].year) + "-12-31", freq="5D", calendar="noleap"))
    paths = [str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-shear/" + year + "/CREG12.L75-FUT08_y" 
                 + "%04d" % d.year + "m" + "%02d" % d.month + "d" + "%02d" % d.day + ".5d_shear.nc") for d in dates]
    xr.save_mfdataset(datasets, paths, engine="netcdf4", format="NETCDF4")

In [ ]:
print("clim")
data_filesFUT_gridU = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m*_gridUclim.nc"))
data_filesFUT_gridV = sorted(glob("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-UV_clim/CREG12.L75-FUT08_m*_gridVclim.nc"))
UFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridU, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
VFUT = xr.decode_cf(xr.open_mfdataset(data_filesFUT_gridV, coords="minimal", compat="override", parallel=True, decode_cf=False, combine="nested", concat_dim="time_counter"))
uUFUT = UFUT.vozocrtx.interp(x=np.arange(0.5, 1580, 1)).rename({"depthu": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
vVFUT = VFUT.vomecrty.interp(y=np.arange(0.5, 1801, 1)).rename({"depthv": "deptht"}).assign_coords({"x": np.arange(0, 1580), "y": np.arange(0, 1801)})
dudzFUT = uUFUT.differentiate("deptht")
dvdzFUT = vVFUT.differentiate("deptht")
shearFUT = ((dudzFUT**2) + (dvdzFUT**2)).to_dataset(name="vovshear")
path = str("/data0/project/drakkar/USERS/jrieck/CREG12.L75-FUT08-shear/clim/CREG12.L75-FUT08_2051-2070_climatology_vovshear.nc")
shearFUT.to_netcdf(path, engine="netcdf4", format="NETCDF4")